In [1]:
import argparse
import pathlib
import sys

import pandas as pd
from cytotable import convert, presets

sys.path.append("../../../utils")
import uuid

import duckdb
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

In [2]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="Patient ID to process, e.g. 'P01'",
    )
    argparser.add_argument(
        "--well_fov",
        type=str,
        required=True,
        help="Well and field of view to process, e.g. 'A01_1'",
    )
    args = argparser.parse_args()
    patient = args.patient
    well_fov = args.well_fov
else:
    patient = "NF0014"
    well_fov = "C4-1"

In [3]:
input_sqlite_file = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/0.converted_profiles/{well_fov}/{well_fov}.duckdb"
).resolve(strict=True)
destination_sc_parquet_file = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/0.converted_profiles/{well_fov}/sc_profiles_{well_fov}.parquet"
).resolve()
destination_organoid_parquet_file = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/0.converted_profiles/{well_fov}/organoid_profiles_{well_fov}.parquet"
).resolve()
destination_sc_parquet_file.parent.mkdir(parents=True, exist_ok=True)
dest_datatype = "parquet"

In [4]:
# show the tables
with duckdb.connect(input_sqlite_file) as con:
    tables = con.execute("SHOW TABLES").fetchdf()
    print(tables)
    nuclei_table = con.sql("SELECT * FROM Nuclei").df()
    cells_table = con.sql("SELECT * FROM Cell").df()
    cytoplasm_table = con.sql("SELECT * FROM Cytoplasm").df()
    organoid_table = con.sql("SELECT * FROM Organoid").df()

        name
0       Cell
1  Cytoplasm
2     Nuclei
3   Organoid


In [5]:
nuclei_id_set = set(nuclei_table["object_id"].to_list())
cells_id_set = set(cells_table["object_id"].to_list())
cytoplasm_id_set = set(cytoplasm_table["object_id"].to_list())
# find the intersection of the three sets
intersection_set = nuclei_id_set.intersection(cells_id_set, cytoplasm_id_set)
# keep only the rows in the three tables that are in the intersection set
nuclei_table = nuclei_table[nuclei_table["object_id"].isin(intersection_set)]
cells_table = cells_table[cells_table["object_id"].isin(intersection_set)]
cytoplasm_table = cytoplasm_table[cytoplasm_table["object_id"].isin(intersection_set)]

In [6]:
# connect to DuckDB and register the tables
with duckdb.connect() as con:
    con.register("df1", nuclei_table)
    con.register("df2", cells_table)
    con.register("df3", cytoplasm_table)
    # Merge them with SQL
    merged_df = con.execute("""
        SELECT *
        FROM df1
        LEFT JOIN df2 USING (object_id)
        LEFT JOIN df3 USING (object_id)
    """).df()

In [7]:
# save the organoid data as parquet
print(f"Final organoid data shape: {merged_df.shape}")
organoid_table.to_parquet(destination_organoid_parquet_file, index=False)
organoid_table.head()

Final organoid data shape: (24, 1926)


,object_id,image_set,Colocalization_Organoid_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MEDIAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MAX.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MEAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.BF_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.BF_MIN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.BF_MAX.MANDERS.COEFF.M1,...,Area.Size.Shape_Organoid_MIN.X,Area.Size.Shape_Organoid_MAX.X,Area.Size.Shape_Organoid_MIN.Y,Area.Size.Shape_Organoid_MAX.Y,Area.Size.Shape_Organoid_MIN.Z,Area.Size.Shape_Organoid_MAX.Z,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Area.Size.Shape_Organoid_SURFACE.AREA
0,59,C4-1,0.005862,0.005862,0.005862,0.005862,1.0,1.0,1.0,1.0,...,128.0,1195.0,26.0,1500.0,12.0,45.0,0.500953,2.0,367.557129,41493.03125


In [8]:
print(f"Final merged single cell dataframe shape: {merged_df.shape}")
# save the sc data as parquet
merged_df.to_parquet(destination_sc_parquet_file, index=False)
merged_df.head()

Final merged single cell dataframe shape: (24, 1926)


,object_id,image_set,Colocalization_Nuclei_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.BF_MEDIAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.BF_MIN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.BF_MAX.CORRELATION.COEFF,Colocalization_Nuclei_AGP.BF_MEAN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.BF_MEDIAN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.BF_MIN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.BF_MAX.MANDERS.COEFF.M1,...,Area.Size.Shape_Cytoplasm_MIN.X,Area.Size.Shape_Cytoplasm_MAX.X,Area.Size.Shape_Cytoplasm_MIN.Y,Area.Size.Shape_Cytoplasm_MAX.Y,Area.Size.Shape_Cytoplasm_MIN.Z,Area.Size.Shape_Cytoplasm_MAX.Z,Area.Size.Shape_Cytoplasm_EXTENT,Area.Size.Shape_Cytoplasm_EULER.NUMBER,Area.Size.Shape_Cytoplasm_EQUIVALENT.DIAMETER,Area.Size.Shape_Cytoplasm_SURFACE.AREA
0,37,C4-1,0.000686,0.000686,0.000686,0.000686,0.999999,0.999999,0.999999,0.999999,...,776.0,834.0,397.0,505.0,0.0,21.0,0.217600,-3.0,37.952808,232.677429
1,43,C4-1,-0.006356,-0.006356,-0.006356,-0.006356,1.000000,1.000000,1.000000,1.000000,...,733.0,962.0,821.0,1016.0,8.0,45.0,0.126108,-19.0,73.553825,1815.736084
2,49,C4-1,0.038544,0.038544,0.038544,0.038544,1.000000,1.000000,1.000000,1.000000,...,721.0,866.0,574.0,727.0,0.0,38.0,0.161503,-43.0,63.827595,1938.586792
3,55,C4-1,0.003026,0.003026,0.003026,0.003026,1.000000,1.000000,1.000000,1.000000,...,599.0,883.0,1029.0,1294.0,0.0,28.0,0.160346,-25.0,86.415962,3007.956299
4,62,C4-1,-0.003472,-0.003472,-0.003472,-0.003472,1.000000,1.000000,1.000000,1.000000,...,339.0,495.0,1067.0,1225.0,12.0,45.0,0.217871,-42.0,69.689224,3012.847656
